In [ ]:
# Import necessary libraries
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import csv
from opencage.geocoder import OpenCageGeocode

In [ ]:
key = '114a46da8f394ca388bb95d41e814ecf'
geocoder = OpenCageGeocode(key)

In [ ]:
# Declare input and output CSV files
city_file = "cities.csv"
title_file = "titles.csv"
output_file = 'output_file.csv'

In [ ]:
# Read titles file and replace spaces with "-"
title_data = pd.read_csv(title_file, encoding="ISO-8859-1")
title_data['NewTitle']=title_data.JobTitle.replace(' ','-',regex=True) 
NewTitles = title_data['NewTitle'].tolist()
NewTitles
title_data.tail()

In [ ]:
# Store all cities in a list in City+State format
city_data = pd.read_csv(city_file, encoding="ISO-8859-1")
city_data['CityState']=city_data.city.replace(' ','-',regex=True)+'--'+city_data.statecode
Cities = city_data['CityState'].tolist()
Cities

In [ ]:
# Open the output CSV file and Write the header row first
with open(output_file, 'w', newline = '') as f:
    thewriter = csv.writer(f)
    thewriter.writerow(['Job Title', 'City', 'State', 'Latitude', 'Longitude', 'Average Salary', 'Job Openings', 
                        'Salary URL', 'Job Openings URL'])
    
    # Webscraping - for each title + city, get the average salary, number of job openings 
    for CityState in Cities:
            
        #Get Lat and Long
        results = geocoder.geocode(CityState)
        lat = results[0]['geometry']['lat']
        long = results[0]['geometry']['lng']

        for NewTitle in NewTitles:       

            try:
                URL_salary = "https://www.indeed.com/career/" + NewTitle + "/salaries/" + CityState
                print(URL_salary)
                page = requests.get(URL_salary)
                soup = BeautifulSoup(page.text, "html.parser")
                salary = soup.find(name="div", class_="sal-agg-nonbase__average-salary-value" ).text;
            except:
                salary = '0';

            try:
                URL_jobs = "https://www.indeed.com/jobs?q=" + NewTitle + "&l=" + CityState 
                print(URL_jobs)
                page = requests.get(URL_jobs)
                soup = BeautifulSoup(page.text, "html.parser")
                jobs = soup.find(name="div", id="searchCountPages").text;
                numjobs = jobs.split()[3]
            except:
                numjobs = '0';


            print(NewTitle)
            jobtitle = NewTitle.replace('-',' ')

            cityname = CityState.split('--')[0].replace('-',' ')
            statecode= CityState.split('--')[1]
               
            # For each city+state, write a row in the CSV file
            thewriter.writerow([jobtitle, cityname, statecode, lat, long, salary, numjobs, URL_salary, URL_jobs])  
output_file.close()